# K-Anonimato com Generalização

## Importações e definições de variáveis/funções globais

In [1]:
import numpy as np
import pandas as pd
import itertools as it
from datetime import datetime as dt
import re
from IPython.display import clear_output

filename        = "Dataset_Covid_CE.csv"
k_values        = [2, 4, 8, 16]

df_municipios = pd.read_csv('ce-regions-format.csv')
municipios_dict = df_municipios.set_index('municipioCaso').to_dict()
municipios = df_municipios['municipioCaso'].unique().tolist()
regions = df_municipios['regiaoPlanejamentoCaso'].unique().tolist()

def get_filename_out(k):
    return f"Dataset_Covid_CE_Anon_Gen_k{k}.csv"

def generate_csv(df, k):
    return df.to_csv(get_filename_out(k), index=False)

def is_k_anon(df, semi_ids, k=2):
    """
    para a tabela ser considerada k-anonima, todo elemento da tabela deve possuir pelo menos k outros elementos com a mesma combinaçao de semi-identificadores. ou seja, para k = 2 e semi_ids = {Atr1, Atr2}, todo registro da tabela deve aparecer pelo menos 2 vezes com mesmo valor nos atributos "Atr1" e "Atr2".

    essa função agrupa os elementos da tabela pelos atributos em semi_ids e retorna False se alguma combinação ocorre menos que k vezes, True se toda combinação ocorre ao menos k vezes.
    """

    groups = df.groupby(by=semi_ids).size()
    for group_size in groups:
        if group_size < k: 
            return False
    return True


## Leitura e limpeza do dataset inicial
Fazemos apenas uma limpeza básica para não modificar muito os resultados do algoritmo.

In [2]:
# definição dos semi-identificadores
semi_ids = [
    "municipioCaso",
    "sexoCaso",
    "dataNascimento",
    "resultadoFinalExame",
    "racaCor",
]

# definição dos atributos sensíveis
sensitive = [
    "comorbidadeCardiovascularSivep",
    "comorbidadeDiabetesSivep",
]

# definição dos tipos
dtype = {
    "municipioCaso"                  : "str",
    "sexoCaso"                       : "str",
    "dataNascimento"                 : "str",
    "resultadoFinalExame"            : "str",
    "comorbidadeCardiovascularSivep" : "str",
    "comorbidadeDiabetesSivep"       : "str",
    "racaCor"                        : "str",
}

# definição das colunas de data
date_columns = [
    "dataNascimento",
]
date_parser = lambda x: pd.to_datetime(x, format="%Y-%m-%d", errors = 'coerce')

# pegar apenas os atributos desejados do dataset
df = pd.read_csv(filename, usecols=semi_ids + sensitive, dtype=dtype)
# df = pd.read_csv(filename, nrows=500000, usecols=semi_ids + sensitive, dtype=dtype)
df = df.dropna(how="all") # remover registros com todos os valores nulo
df.racaCor.replace(np.nan, 'Sem Informacao', inplace=True)
df = df.fillna("*") # padroniza valores nulos
df


,municipioCaso,sexoCaso,dataNascimento,resultadoFinalExame,comorbidadeCardiovascularSivep,comorbidadeDiabetesSivep,racaCor
19,SOBRAL,MASCULINO,2003-08-14,Negativo,*,*,Parda
20,PACAJUS,MASCULINO,1983-11-07,Negativo,*,*,Parda
21,HORIZONTE,FEMININO,1982-01-14,Negativo,*,*,Sem Informacao
25,FORTALEZA,MASCULINO,1992-03-12,Negativo,*,*,Parda
28,CAUCAIA,MASCULINO,1970-03-06,Negativo,*,*,Sem Informacao
...,...,...,...,...,...,...,...
1266272,FORTALEZA,FEMININO,1982-12-08,Positivo,*,*,Branca
1266273,FORTALEZA,FEMININO,1980-12-02,Negativo,*,*,Parda
1266274,FORTALEZA,FEMININO,1988-05-17,Provável,*,*,Branca
1266275,CRATEUS,FEMININO,1970-09-28,Negativo,*,*,Parda


In [3]:
df.describe()

,municipioCaso,sexoCaso,dataNascimento,resultadoFinalExame,comorbidadeCardiovascularSivep,comorbidadeDiabetesSivep,racaCor
count,911921,911921,911921,911921,911921,911921,911921
unique,184,3,35436,6,4,4,7
top,FORTALEZA,FEMININO,*,Negativo,*,*,Parda
freq,238374,517751,206,543568,902505,903015,482975


In [4]:
df.sexoCaso.value_counts()


FEMININO     517751
MASCULINO    392647
*              1523
Name: sexoCaso, dtype: int64

## Definição do algoritmo


In [5]:
def k_anon_general(df, semi_ids, k, dropAnonLevel = True):
    """
    algoritmo de generalização.

    funcionamento:
    1. agrupar registros pelos semi-identificadores
    2. realizar generalização dos atributos de grupos com tamanho < k
    2.5 atributos são generalizados com base na quantidade de valores únicos, de maior para o menor
    3. repetir até possuir < k número de registros não-anonimados
    4. puxar dos registros já anonimados k - n registros para novamente realizar generalização, até todo grupo formado possuir tamanho >= k

    df -- dataframe para ser anonimado
    semi_ids -- colunas de attr semi identificadores
    k -- valor de k
    dropAnonLevel -- se a coluna que representa o grau de generalização do registro deve permanecer no dataframe anonimado

    return -- dataframe anonimado para valor de k
    """

    # df deve possuir pelo menos k registros
    if df.shape[0] < k:
        return None

    print('iniciando generalização')
    cdf = df.copy()

    # cria coluna para salvar grau de generalização
    cdf['anonLevel'] = 0

    # enquanto a tabela não respeitar o k-anonimato
    while not is_k_anon(cdf, semi_ids, k):
        # forma grupos com combinações de valores dos semi-ids
        groups = cdf.groupby(semi_ids)
        # print(f'{groups.count()} combinações formadas')

        # pega registros pertencendo a grupos que não cumprem k-anonimato
        not_anon = cdf[groups['comorbidadeDiabetesSivep'].transform('count').lt(k)].copy()
        # not_anon = cdf[cdf[semi_ids].map(cdf.groupby(semi_ids)['comorbidadeCardiovascularSivep'].count()).lt(k)]

        # se sobrarem menos que k registros que ainda não estão anonimados, pegar registros de grupos já formados para generalizar novamente e criar grupo com os registros faltantes
        if not_anon.shape[0] < k:
            # pegar k - qnt. de registros faltantes registros com maior grau de generalização
            these_too = cdf.drop(not_anon.index).loc[cdf['anonLevel'] >= not_anon['anonLevel'].min()-1].nlargest(k-not_anon.shape[0], 'anonLevel')
            not_anon = pd.concat([not_anon, these_too])
            # return these_too
        
        print(f'{not_anon.shape[0]} registros precisam ser anonimados')

        # decide qual atributo deve ser generalizado, com base na quantidade de valores únicos daquele atributo
        max_freq_col = not_anon[semi_ids].nunique().idxmax(axis=1)
        print(f'generalizando coluna {max_freq_col}')

        # definição das funções de generalização para cada semi_id
        callback = None
        if max_freq_col == 'dataNascimento':
            def gen_data(value):
                """generaliza na ordem 0: ano-mes-dia -> 1: ano-mes -> 2: ano -> 3: ano* (ex: 201*) -> 4: ano** (ex: 20**) -> 5: *"""

                reg = re.search(r'^([0-9]{4})-([0-9]{1,2})-([0-9]{1,2})$', value)
                if reg:
                    return f'{reg.group(1)}-{reg.group(2)}'

                reg = re.search(r'^([0-9]{4})-([0-9]{1,2})$', value)
                if reg:
                    return f'{reg.group(1)}'
                
                if re.match(r'^[0-9]{4}$', value):
                    return f'{value[:-1]}*'
                
                if re.match(r'^[0-9]{3}\*$', value):
                    return f'{value[:-2]}**'

                else:
                    return '*'
            callback = gen_data

        elif max_freq_col == 'municipioCaso':
            def gen_municipios(value):
                """generaliza na ordem 0: municipio -> 1: região -> 2: estado -> 3: *"""

                if value in municipios:
                    return municipios_dict['regiaoPlanejamentoCaso'][value]
                elif value in regions:
                    return 'CEARA'
                else:
                    return '*'
            callback = gen_municipios

        elif max_freq_col == 'resultadoFinalExame':
            def gen_resultados(value):
                """generaliza na ordem 0: resultado -> 1: conhecido | desconhecido -> 2: *"""

                if value == 'Conhecido' or value == 'Desconhecido' or value == '*':
                    return '*'
                elif value == 'Positivo' or value == 'Negativo':
                    return 'Conhecido'
                else: 
                    return 'Desconhecido'
            callback = gen_resultados
            
        elif max_freq_col == 'racaCor':
            def gen_race(value):
                """generaliza na ordem 0: cor -> 1: humano -> 2: *"""

                if value == 'Humano' or value == '*':
                    return '*'
                else: 
                    return 'Humano'
            callback = gen_race
            
        elif max_freq_col == 'sexoCaso':
            def gen_sexo(value):
                """generaliza na ordem 0: genero -> 1: binário -> 2: *"""

                if value == 'BINARIO' or value == '*':
                    return '*'
                else: 
                    return 'BINARIO'
            callback = gen_sexo
        
        # aplica generalização escolhida nos registros
        cdf.loc[not_anon.index, max_freq_col] = cdf.loc[not_anon.index, max_freq_col].map(callback)

        # aumenta contador do nível de anonimização
        # min = 0, max = 14
        cdf.loc[not_anon.index, 'anonLevel'] = cdf.loc[not_anon.index, 'anonLevel'].map(lambda x: min(max(0, x + 1), 14))
    
    if (dropAnonLevel):
        return cdf.drop(columns=['anonLevel'])
    return cdf


## Aplicação do algoritmo
Agora aplicamos o algoritmo para os valores de k = {2, 4, 8, 16}

In [6]:
k = 2
df2 = k_anon_general(df, semi_ids, k, False)
df2


iniciando generalização
642864 registros precisam ser anonimados
generalizando coluna dataNascimento
321014 registros precisam ser anonimados
generalizando coluna dataNascimento
84383 registros precisam ser anonimados
generalizando coluna municipioCaso
16349 registros precisam ser anonimados
generalizando coluna dataNascimento
1951 registros precisam ser anonimados
generalizando coluna dataNascimento
691 registros precisam ser anonimados
generalizando coluna municipioCaso
79 registros precisam ser anonimados
generalizando coluna racaCor
24 registros precisam ser anonimados
generalizando coluna dataNascimento
11 registros precisam ser anonimados
generalizando coluna resultadoFinalExame
7 registros precisam ser anonimados
generalizando coluna resultadoFinalExame


,municipioCaso,sexoCaso,dataNascimento,resultadoFinalExame,comorbidadeCardiovascularSivep,comorbidadeDiabetesSivep,racaCor,anonLevel
19,SOBRAL,MASCULINO,2003-08,Negativo,*,*,Parda,1
20,PACAJUS,MASCULINO,1983-11-07,Negativo,*,*,Parda,0
21,HORIZONTE,FEMININO,1982,Negativo,*,*,Sem Informacao,2
25,FORTALEZA,MASCULINO,1992-03-12,Negativo,*,*,Parda,0
28,CAUCAIA,MASCULINO,1970-03,Negativo,*,*,Sem Informacao,1
...,...,...,...,...,...,...,...,...
1266272,FORTALEZA,FEMININO,1982-12,Positivo,*,*,Branca,1
1266273,FORTALEZA,FEMININO,1980-12-02,Negativo,*,*,Parda,0
1266274,FORTALEZA,FEMININO,1988-05,Provável,*,*,Branca,1
1266275,CRATEUS,FEMININO,1970-09-28,Negativo,*,*,Parda,0


In [7]:
is_k_anon(df2, semi_ids, k)


True

In [8]:
k = 4
df4 = k_anon_general(df, semi_ids, k, False)
df4


iniciando generalização
855392 registros precisam ser anonimados
generalizando coluna dataNascimento
550074 registros precisam ser anonimados
generalizando coluna dataNascimento
187881 registros precisam ser anonimados
generalizando coluna municipioCaso
39223 registros precisam ser anonimados
generalizando coluna dataNascimento
4428 registros precisam ser anonimados
generalizando coluna dataNascimento
1726 registros precisam ser anonimados
generalizando coluna municipioCaso
161 registros precisam ser anonimados
generalizando coluna racaCor
61 registros precisam ser anonimados
generalizando coluna dataNascimento
19 registros precisam ser anonimados
generalizando coluna resultadoFinalExame
15 registros precisam ser anonimados
generalizando coluna sexoCaso
5 registros precisam ser anonimados
generalizando coluna municipioCaso
5 registros precisam ser anonimados
generalizando coluna sexoCaso
4 registros precisam ser anonimados
generalizando coluna resultadoFinalExame
4 registros precisam s

,municipioCaso,sexoCaso,dataNascimento,resultadoFinalExame,comorbidadeCardiovascularSivep,comorbidadeDiabetesSivep,racaCor,anonLevel
19,SOBRAL,MASCULINO,2003,Negativo,*,*,Parda,2
20,PACAJUS,MASCULINO,1983-11,Negativo,*,*,Parda,1
21,HORIZONTE,FEMININO,1982,Negativo,*,*,Sem Informacao,2
25,FORTALEZA,MASCULINO,1992-03,Negativo,*,*,Parda,1
28,CAUCAIA,MASCULINO,1970,Negativo,*,*,Sem Informacao,2
...,...,...,...,...,...,...,...,...
1266272,FORTALEZA,FEMININO,1982-12,Positivo,*,*,Branca,1
1266273,FORTALEZA,FEMININO,1980-12,Negativo,*,*,Parda,1
1266274,FORTALEZA,FEMININO,1988-05,Provável,*,*,Branca,1
1266275,CRATEUS,FEMININO,1970-09-28,Negativo,*,*,Parda,0


In [9]:
is_k_anon(df4, semi_ids, k)


True

In [10]:
k = 8
df8 = k_anon_general(df, semi_ids, k, False)
df8


iniciando generalização
903304 registros precisam ser anonimados
generalizando coluna dataNascimento
699113 registros precisam ser anonimados
generalizando coluna dataNascimento
322833 registros precisam ser anonimados
generalizando coluna municipioCaso
77252 registros precisam ser anonimados
generalizando coluna dataNascimento
9333 registros precisam ser anonimados
generalizando coluna dataNascimento
3425 registros precisam ser anonimados
generalizando coluna municipioCaso
367 registros precisam ser anonimados
generalizando coluna racaCor
127 registros precisam ser anonimados
generalizando coluna dataNascimento
45 registros precisam ser anonimados
generalizando coluna resultadoFinalExame
23 registros precisam ser anonimados
generalizando coluna sexoCaso
10 registros precisam ser anonimados
generalizando coluna resultadoFinalExame
10 registros precisam ser anonimados
generalizando coluna municipioCaso
10 registros precisam ser anonimados
generalizando coluna sexoCaso


,municipioCaso,sexoCaso,dataNascimento,resultadoFinalExame,comorbidadeCardiovascularSivep,comorbidadeDiabetesSivep,racaCor,anonLevel
19,SOBRAL,MASCULINO,2003,Negativo,*,*,Parda,2
20,PACAJUS,MASCULINO,1983,Negativo,*,*,Parda,2
21,GRANDE FORTALEZA,FEMININO,1982,Negativo,*,*,Sem Informacao,3
25,FORTALEZA,MASCULINO,1992-03,Negativo,*,*,Parda,1
28,CAUCAIA,MASCULINO,1970,Negativo,*,*,Sem Informacao,2
...,...,...,...,...,...,...,...,...
1266272,FORTALEZA,FEMININO,1982,Positivo,*,*,Branca,2
1266273,FORTALEZA,FEMININO,1980-12,Negativo,*,*,Parda,1
1266274,FORTALEZA,FEMININO,1988-05,Provável,*,*,Branca,1
1266275,CRATEUS,FEMININO,1970,Negativo,*,*,Parda,2


In [11]:
is_k_anon(df8, semi_ids, k)


True

In [12]:
k = 16
df16 = k_anon_general(df, semi_ids, k, False)
df16


iniciando generalização
910607 registros precisam ser anonimados
generalizando coluna dataNascimento
793970 registros precisam ser anonimados
generalizando coluna dataNascimento
459870 registros precisam ser anonimados
generalizando coluna municipioCaso
138196 registros precisam ser anonimados
generalizando coluna dataNascimento
18914 registros precisam ser anonimados
generalizando coluna dataNascimento
6555 registros precisam ser anonimados
generalizando coluna municipioCaso
568 registros precisam ser anonimados
generalizando coluna racaCor
309 registros precisam ser anonimados
generalizando coluna dataNascimento
84 registros precisam ser anonimados
generalizando coluna resultadoFinalExame
21 registros precisam ser anonimados
generalizando coluna sexoCaso
21 registros precisam ser anonimados
generalizando coluna resultadoFinalExame
21 registros precisam ser anonimados
generalizando coluna municipioCaso
16 registros precisam ser anonimados
generalizando coluna sexoCaso
16 registros pre

,municipioCaso,sexoCaso,dataNascimento,resultadoFinalExame,comorbidadeCardiovascularSivep,comorbidadeDiabetesSivep,racaCor,anonLevel
19,SOBRAL,MASCULINO,2003,Negativo,*,*,Parda,2
20,GRANDE FORTALEZA,MASCULINO,1983,Negativo,*,*,Parda,3
21,GRANDE FORTALEZA,FEMININO,1982,Negativo,*,*,Sem Informacao,3
25,FORTALEZA,MASCULINO,1992-03,Negativo,*,*,Parda,1
28,CAUCAIA,MASCULINO,1970,Negativo,*,*,Sem Informacao,2
...,...,...,...,...,...,...,...,...
1266272,FORTALEZA,FEMININO,1982,Positivo,*,*,Branca,2
1266273,FORTALEZA,FEMININO,1980-12,Negativo,*,*,Parda,1
1266274,FORTALEZA,FEMININO,1988,Provável,*,*,Branca,2
1266275,CRATEUS,FEMININO,1970,Negativo,*,*,Parda,2


In [13]:
is_k_anon(df16, semi_ids, k)

True

## Gerar CSVs

In [14]:
generate_csv(df, k=1)
generate_csv(df2, k=2)
generate_csv(df4, k=4)
generate_csv(df8, k=8)
generate_csv(df16, k=16)
